In [185]:
import pandas as pd
import numpy as np
import configparser
from category_encoders import CountEncoder


from sklearn.model_selection import train_test_split

In [186]:
pip install --upgrade category_encoders


Note: you may need to restart the kernel to use updated packages.


In [187]:
pip install category_encoders


Note: you may need to restart the kernel to use updated packages.


In [188]:
dataset = pd.read_csv('../data/raw/train.csv')
dataset.head()

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction
0,1457,34,Female,138842,Weekly,162.954929,Indian,Dinner,8,Celebration,Dine-in,1,0,1,4.228618,1,5,3,1
1,1371,54,Male,98671,Rarely,66.918873,Indian,Lunch,4,Casual,Takeaway,0,0,0,50.247186,1,2,2,0
2,1505,47,Male,122351,Monthly,94.127670,Indian,Lunch,7,Casual,Takeaway,0,1,0,10.174873,3,3,5,0
3,1011,38,Male,78868,Weekly,92.705568,Mexican,Dinner,4,Business,Takeaway,0,0,1,14.237746,4,1,4,0
4,1016,50,Female,128686,Monthly,166.931144,American,Dinner,8,Business,Dine-in,0,0,0,56.319628,4,3,4,0


In [189]:
print(dataset.dtypes)

CustomerID                int64
Age                       int64
Gender                   object
Income                    int64
VisitFrequency           object
AverageSpend            float64
PreferredCuisine         object
TimeOfVisit              object
GroupSize                 int64
DiningOccasion           object
MealType                 object
OnlineReservation         int64
DeliveryOrder             int64
LoyaltyProgramMember      int64
WaitTime                float64
ServiceRating             int64
FoodRating                int64
AmbianceRating            int64
HighSatisfaction          int64
dtype: object


In [190]:
config = configparser.ConfigParser()
config.read('../pipeline.cfg')
valor = config.get('GENERAL', 'VARS_TO_DROP')
valor2 = config.get('CATEGORICAL', 'LABEL_ENCODER')
print(valor)
print(valor2)

CustomerID
Gender,VisitFrequency,PreferredCuisine,TimeOfVisit,DiningOccasion,MealType


In [191]:
x_feature = dataset.drop(labels=list(config.get('GENERAL', 'VARS_TO_DROP').split(',')), axis=1)
y_target = dataset[config.get('GENERAL', 'TARGET')]
x_train, x_test, y_train, y_test = train_test_split(x_feature, y_target, test_size=0.3, shuffle=True, random_state=2025)

## de nuevo

In [192]:
print(dataset.columns)


Index(['CustomerID', 'Age', 'Gender', 'Income', 'VisitFrequency',
       'AverageSpend', 'PreferredCuisine', 'TimeOfVisit', 'GroupSize',
       'DiningOccasion', 'MealType', 'OnlineReservation', 'DeliveryOrder',
       'LoyaltyProgramMember', 'WaitTime', 'ServiceRating', 'FoodRating',
       'AmbianceRating', 'HighSatisfaction'],
      dtype='object')


In [193]:
cols_to_drop = [col.strip() for col in config.get('GENERAL', 'VARS_TO_DROP').split(',')]


In [194]:
if 'CostumerID' in dataset.columns:
    print("La columna 'CostumerID' está presente")
else:
    print("La columna 'CostumerID' no está presente")


La columna 'CostumerID' no está presente


In [195]:
columns_to_drop = list(config.get('GENERAL', 'VARS_TO_DROP').split(','))
# Verificar si la columna está en el dataset
for col in columns_to_drop:
    if col in dataset.columns:
        print(f"La columna '{col}' está presente.")
    else:
        print(f"La columna '{col}' no está presente.")

# Eliminar las columnas si existen
x_feature = dataset.drop(labels=columns_to_drop, axis=1)


La columna 'CustomerID' está presente.


In [196]:
cols_to_drop = [col.strip() for col in config.get('GENERAL', 'VARS_TO_DROP').split(',')]
cols_to_drop = [col for col in cols_to_drop if col in dataset.columns]  # Verifica que la columna exista

x_feature = dataset.drop(labels=cols_to_drop, axis=1)


In [197]:
x_feature = dataset.drop(labels=list(config.get('GENERAL', 'VARS_TO_DROP').split(',')), axis=1)


In [198]:
import pandas as pd
import numpy as np
import configparser
from sklearn.model_selection import train_test_split

# Leer el dataset
dataset = pd.read_csv('../data/raw/train.csv')

# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('../pipeline.cfg')

# Obtener columnas a eliminar de la configuración y limpiar espacios
cols_to_drop = [col.strip() for col in config.get('GENERAL', 'VARS_TO_DROP').split(',')]

# Verificar si las columnas a eliminar están en el DataFrame
cols_to_drop = [col for col in cols_to_drop if col in dataset.columns]

# Eliminar las columnas que existen
x_feature = dataset.drop(labels=cols_to_drop, axis=1)

# Obtener la variable objetivo
y_target = dataset[config.get('GENERAL', 'TARGET')]

# Dividir el dataset en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x_feature, y_target, test_size=0.3, shuffle=True, random_state=2025)


In [199]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Definir las columnas numéricas y categóricas
categorical_columns = ['Gender', 'VisitFrequency', 'PreferredCuisine', 'TimeOfVisit', 'DiningOccasion', 'MealType']
numeric_columns = ['Age', 'Income', 'AverageSpend', 'GroupSize', 'WaitTime', 'ServiceRating', 'FoodRating', 'AmbianceRating']

# Crear el preprocesador con imputación y codificación para columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_columns),  # Imputación de columnas numéricas
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación para valores nulos en categóricos
            ('encoder', OneHotEncoder(handle_unknown='ignore'))   # Codificación OneHot para las columnas categóricas
        ]), categorical_columns)  # Aplicar a las columnas categóricas
    ])

# Definir el pipeline con el preprocesador y el clasificador
restaurante_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=2025))
])

# Ajustar el modelo con los datos de entrenamiento
restaurante_model.fit(x_train, y_train)

# Hacer predicciones
y_pred = restaurante_model.predict(x_test)
print(f"Predicciones: {y_pred[:10]}")


Predicciones: [1 0 1 1 0 1 0 1 1 1]


In [200]:
# Revisar los valores únicos en las columnas categóricas
for column in categorical_columns:
    print(f"Valores únicos en {column}: {dataset[column].unique()}")


Valores únicos en Gender: ['Female' 'Male']
Valores únicos en VisitFrequency: ['Weekly' 'Rarely' 'Monthly' 'Daily']
Valores únicos en PreferredCuisine: ['Indian' 'Mexican' 'American' 'Chinese' 'Italian']
Valores únicos en TimeOfVisit: ['Dinner' 'Lunch' 'Breakfast']
Valores únicos en DiningOccasion: ['Celebration' 'Casual' 'Business']
Valores únicos en MealType: ['Dine-in' 'Takeaway']


In [201]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Definir las columnas numéricas y categóricas
categorical_columns = ['Gender', 'VisitFrequency', 'PreferredCuisine', 'TimeOfVisit', 'DiningOccasion', 'MealType']
numeric_columns = ['Age', 'Income', 'AverageSpend', 'GroupSize', 'WaitTime', 'ServiceRating', 'FoodRating', 'AmbianceRating']

# Crear el preprocesador con imputación y codificación para columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_columns),  # Imputación de columnas numéricas
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación para valores nulos en categóricos
            ('encoder', OneHotEncoder(handle_unknown='ignore'))   # Codificación OneHot para las columnas categóricas
        ]), categorical_columns)  # Aplicar a las columnas categóricas
    ])

# Definir el pipeline con el preprocesador y el clasificador
restaurante_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=2025))
])

# Ajustar el modelo con los datos de entrenamiento
restaurante_model.fit(x_train, y_train)

# Hacer predicciones
y_pred = restaurante_model.predict(x_test)
print(f"Predicciones: {y_pred[:10]}")


Predicciones: [1 0 1 1 0 1 0 1 1 1]


In [202]:
# Revisar los valores únicos en las columnas categóricas
for column in categorical_columns:
    print(f"Valores únicos en {column}: {dataset[column].unique()}")


Valores únicos en Gender: ['Female' 'Male']
Valores únicos en VisitFrequency: ['Weekly' 'Rarely' 'Monthly' 'Daily']
Valores únicos en PreferredCuisine: ['Indian' 'Mexican' 'American' 'Chinese' 'Italian']
Valores únicos en TimeOfVisit: ['Dinner' 'Lunch' 'Breakfast']
Valores únicos en DiningOccasion: ['Celebration' 'Casual' 'Business']
Valores únicos en MealType: ['Dine-in' 'Takeaway']


In [203]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Definir las columnas numéricas y categóricas
categorical_columns = ['Gender', 'VisitFrequency', 'PreferredCuisine', 'TimeOfVisit', 'DiningOccasion', 'MealType']
numeric_columns = ['Age', 'Income', 'AverageSpend', 'GroupSize', 'WaitTime', 'ServiceRating', 'FoodRating', 'AmbianceRating']

# Crear el preprocesador con imputación y codificación para columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_columns),  # Imputación de columnas numéricas
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación para valores nulos en categóricos
            ('encoder', OneHotEncoder(handle_unknown='ignore'))   # Codificación OneHot para las columnas categóricas
        ]), categorical_columns)  # Aplicar a las columnas categóricas
    ])

# Definir el pipeline con el preprocesador y el clasificador
restaurante_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=2025))
])

# Ajustar el modelo con los datos de entrenamiento
restaurante_model.fit(x_train, y_train)

# Hacer predicciones
y_pred = restaurante_model.predict(x_test)
print(f"Predicciones: {y_pred[:10]}")


Predicciones: [1 0 1 1 0 1 0 1 1 1]


In [204]:
# Definir las columnas
continous_columns = ['Age', 'Income', 'AverageSpend', 'GroupSize', 'WaitTime', 'ServiceRating', 'FoodRating', 'AmbianceRating']
categorical_columns = ['Gender', 'VisitFrequency', 'PreferredCuisine', 'TimeOfVisit', 'DiningOccasion', 'MealType']


In [205]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir las columnas categóricas y continuas
continous_columns = ['Age', 'Income', 'AverageSpend', 'GroupSize', 'WaitTime', 'ServiceRating', 'FoodRating', 'AmbianceRating']
categorical_columns = ['Gender', 'VisitFrequency', 'PreferredCuisine', 'TimeOfVisit', 'DiningOccasion', 'MealType']

# Crear un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        # Imputación de variables continuas (Media)
        ('continues_var_imputation', SimpleImputer(strategy='mean'), continous_columns),
        # Imputación de variables categóricas (Frecuencia más común)
        ('categorical_var_imputacion', SimpleImputer(strategy='most_frequent'), categorical_columns),
        # Codificación de variables categóricas con OneHotEncoder
        ('categorical_encode_ohe', OneHotEncoder(drop='first'), categorical_columns),
        # Escalado de características continuas
        ('feature_scaling', StandardScaler(), continous_columns)
    ]
)

# Función para codificación por frecuencia manual
def frequency_encoding(df, categorical_columns):
    for col in categorical_columns:
        freq_map = df[col].value_counts() / len(df)  # Contar la frecuencia de cada valor
        df[col] = df[col].map(freq_map)  # Sustituir cada valor por su frecuencia
    return df

# Aplicar la codificación por frecuencia
x_feature = frequency_encoding(x_feature, categorical_columns)

# Crear el pipeline con el modelo RandomForest
restaurante_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=2025))
])

# Dividir el dataset en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x_feature, y_target, test_size=0.3, shuffle=True, random_state=2025)

# Ajustar el modelo
restaurante_model.fit(x_train, y_train)

# Hacer predicciones
y_pred = restaurante_model.predict(x_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión: {accuracy:.2f}")

# Evaluar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)


Precisión: 0.96
Matriz de Confusión:
[[273  14]
 [  8 251]]


In [206]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Listas de columnas
continous_columns = ['Age', 'Income', 'AverageSpend', 'GroupSize', 'WaitTime', 'ServiceRating', 'FoodRating', 'AmbianceRating']
categorical_columns = ['Gender', 'VisitFrequency', 'PreferredCuisine', 'TimeOfVisit', 'DiningOccasion', 'MealType']

# Preprocesador con imputación y codificación
preprocessor = ColumnTransformer(
    transformers=[
        ('continues_var_imputation', SimpleImputer(strategy='mean'), continous_columns),
        ('categorical_var_imputacion', SimpleImputer(strategy='most_frequent'), categorical_columns),
        ('categorical_encode_ohe', OneHotEncoder(drop='first'), categorical_columns),
        ('feature_scaling', StandardScaler(), continous_columns)
    ]
)

# Crear el modelo en un pipeline
restaurante_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=2025))
])

# Dividir los datos en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x_feature, y_target, test_size=0.3, shuffle=True, random_state=2025)

# Ajustar el modelo
restaurante_model.fit(x_train, y_train)

# Hacer predicciones
y_pred = restaurante_model.predict(x_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión: {accuracy:.2f}")


Precisión: 0.96


In [208]:
# Mostrar los pasos del pipeline
print(titanic_survived_model.named_steps)


{'preprocessor': ColumnTransformer(transformers=[('continues_var_imputation', SimpleImputer(),
                                 ['Age', 'Income', 'AverageSpend', 'GroupSize',
                                  'WaitTime', 'ServiceRating', 'FoodRating',
                                  'AmbianceRating']),
                                ('categorical_var_imputacion',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['Gender', 'VisitFrequency',
                                  'PreferredCuisine', 'TimeOfVisit',
                                  'DiningOccasion', 'MealType']),
                                ('categorical_encode_ohe',
                                 OneHotEncoder(drop='first'),
                                 ['Gender', 'VisitFrequency',
                                  'PreferredCuisine', 'TimeOfVisit',
                                  'DiningOccasion', 'MealType']),
                                ('feature_

In [211]:
# Realizar predicciones en el conjunto de test
y_pred = restaurante_model.predict(x_test)

# Mostrar las predicciones
print(y_pred)


[1 0 1 1 0 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1
 1 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1
 0 0 1 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 0 1 0
 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 0 1 1 0
 1 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1
 0 1 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 0
 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1
 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 0 1 0
 0 1 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 0 1 1 1 0 1
 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1 0 1 0 0
 1 1 1 0 1 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0
 1 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0
 1 0 0 0 1 1 1 1 0 1 1 0 

In [213]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Suponiendo que y_test contiene las etiquetas verdaderas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar los resultados
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)


Accuracy: 0.9597069597069597
Precision: 0.9471698113207547
Recall: 0.9691119691119691
F1 Score: 0.9580152671755725
Confusion Matrix:
[[273  14]
 [  8 251]]
